In [469]:
### Google Secret Manager Client Instalation


# !pip install google-cloud-secret-manager
# Install gcloud: https://cloud.google.com/sdk/docs/install#windows
# Setup the credential: gcloud auth application-default login

In [470]:
# !pip install aioredis==1.3.1

In [506]:
import json
from google.cloud import secretmanager
import redis
import asyncio
import aioredis

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

In [507]:
def get_secret_value(project_id = "652914548272", secret_id = "redis", key = "redis-test"):
    from google.cloud import secretmanager

    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()
    
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"

    # Access the secret version.
    response = client.access_secret_version(request={"name": name})

    payload = response.payload.data.decode("UTF-8")
    configs = eval(payload)

    config_key = configs[key]

    return config_key


In [508]:
# Redis on Redis Cloud (30mb)
config_redis = get_secret_value(key = "redis-test")

In [510]:
config_redis

{'host': 'redis-11664.c323.us-east-1-2.ec2.cloud.redislabs.com',
 'port': 11664,
 'password': 'fSnTmrFD7TRh5Wvr3hZluWsZyw4Fn13M'}

In [511]:
r = redis.Redis(
  host=config_redis["host"],
  port=config_redis["port"],
  password=config_redis["password"]
)


In [512]:
r.get('meta:document_size')

In [513]:
# r.flushdb()

# Load Index Json

In [325]:
path_index = "../../backend/utils/result/inverted_index.json"
with open(path_index, 'r+') as f:
    data_json = json.load(f)

In [326]:
data_json.keys()

dict_keys(['meta', 'index'])

In [104]:
data_json['meta'].keys()

dict_keys(['document_size', 'doc_ids_list', 'source_doc_ids'])

# Store the metadata of the document

In [464]:
path_index = "../../backend/data/bbc/bbc_data_20240217_301.csv"
df = pd.read_csv(path_index)

In [466]:
# df

# Function Redis

In [378]:
def update_doc_size(r, new_size, colname='document_size'):
    doc_size = r.get(colname)
    
    if not (doc_size):
        doc_size = data_json['meta']['document_size']
    else:
        doc_size = int(doc_size)
        doc_size += data_json['meta']['document_size']

    r.set(colname, doc_size)
    return True

def get_doc_size(r, colname='document_size'):
    doc_size = r.get(colname)
    return int(doc_size)
    

## Store Word Index

### Approach 1, using partial hash

In [406]:
v = data_json['index']['men']['2018']

In [413]:
r.hset('w:men', '2011', str(v))

1

In [410]:
r.hgetall('w:men')

{b'2018': b'[1, 6, 10]'}

In [411]:
r.hexists('w:men', '2018')

True

In [414]:
r.hkeys('w:men')

[b'2018', b'2011']

### Approach 2, using full hash

In [423]:
val = str(data_json['index']['men'])

In [426]:
r.set('w:men-2',val)

True

In [467]:
# r.get('w:men-2')

# Push Index to Redis

In [70]:
# r.set("document_size", str(data_json['document_size']))

In [71]:
r.get('document_size')

b"{'0': 5000}"

In [72]:
# r.set("doc_ids_list", str(data_json['doc_ids_list']))

In [74]:
# r.get('doc_ids_list')

In [75]:
# # Store term positions in documents
# for term in tqdm(data_json.keys()):
#     if term not in ['document_size', 'doc_ids_list']:
#         r.set(term, str(data_json['term'])) #38944/39310

# Push Index to Redis in batches

In [441]:
def process_dict_in_batches(input_dict, batch_size, prefix='w:'):
    keys = list(input_dict.keys())
    num_keys = len(keys)
    batches = []
    for i in range(0, num_keys, batch_size):
        batch_keys = keys[i:i+batch_size]
        batch = {prefix+key: str(input_dict[key]) for key in batch_keys}
        batches.append(batch)
    return batches

In [442]:
minibatch = 15
batches = process_dict_in_batches(data_json['index'], minibatch)

In [444]:
# batches[3]

In [85]:
async def set_data(redis, key, value):
    await redis.set(key, value)

async def batch_push():
    # Define Redis server configuration
    redis_config = {
        'address': (config_redis["host"], config_redis["port"]),  # IP address and port
        'password': config_redis["password"]  # Redis password
    }
    
    # Connect to Redis
    redis = await aioredis.create_redis_pool(**redis_config)

    # Define keys and values to set
    for batch in tqdm(batches):
        # Perform SET operations in parallel
        tasks = [set_data(redis, key, value) for key, value in batch.items()]
        await asyncio.gather(*tasks)

    # Close Redis connection
    redis.close()
    await redis.wait_closed()

# Run the asyncio event loop
# asyncio.run(batch_push()) # use in a script

# await batch_push() # use in a notebook

# Retrieve value from Redis

In [445]:
def get_val(r, key):
    value = r.get(key)
    value = value.decode()
    value = eval(value)
    return value

In [447]:
# get_val(r, 'w:men')

In [78]:
terms = list(data_json.keys())

In [81]:
term = terms[-21]
val = get_val(r, term)

In [82]:
val

{'10037': [911, 912]}

# Insert without overlapping

In [516]:
# Function to insert new data
def insert_data(document_id, document_link):
    # Lua script to check if document link exists in the Redis Set
    lua_script = """
        if redis.call('sismember', KEYS[1], ARGV[1]) == 1 then
            return 'EXIST'
        else
            redis.call('hset', KEYS[2], ARGV[2], ARGV[1])
            redis.call('sadd', KEYS[1], ARGV[1])
            return 'OK'
        end
    """
    # Run the Lua script
    result = r.eval(lua_script, 2, 'document_links', 'documents', document_link, document_id)
    return result

# Function to insert new data
def insert_data(document_id, document_link, title, summary):
    # Lua script to check if document link exists in the Redis Set
    lua_script = """
        if redis.call('sismember', KEYS[1], ARGV[1]) == 1 then
            return 'EXIST'
        else
            redis.call('hset', KEYS[2], ARGV[2], ARGV[1])
            redis.call('hset', KEYS[2], ARGV[3], ARGV[4])
            redis.call('hset', KEYS[2], ARGV[5], ARGV[6])
            redis.call('sadd', KEYS[1], ARGV[1])
            return 'OK'
        end
    """
    # Run the Lua script
    result = r.eval(lua_script, 2, 'document_links', 'documents', document_link, document_id, 'title', title, 'summary', summary)
    return result


def get_data(document_id):
    return r.hget('documents', document_id)

# Function to check if a document ID exists
def document_id_exists(document_id):
    return r.hexists('documents', document_id)

# Function to get the maximum document ID
def get_max_document_id():
    document_ids = r.hkeys('documents')
    if document_ids:
        max_document_id = max(int(document_id) for document_id in document_ids)
        return max_document_id
    else:
        return None



# # Example usage
# document_id = 124
# document_link = 'https://google.com/document'
# result = insert_data(document_id, document_link)
# print(result)

# Example usage
document_id = '122'
document_link = 'https://122.com/document'
title = 'Sample Title22'
summary = 'This is a sample summary.2'
result = insert_data(document_id, document_link, title, summary)
print(result)

b'OK'


In [191]:
def get_document_link(document_id):
    return r.hget('documents', document_id)

document_id = '121'
document_link = get_document_link(document_id)
print("Document Link:", document_link)


Document Link: b'https://2.com/document'


In [192]:
def get_title(document_id):
    return r.hmget('documents', document_id, 'title')[1]

document_id = '122'
title = get_title(document_id)
print("Title:", title)


Title: b'Sample Title22'


In [194]:
def get_all_info(document_id):
    return r.hmget('documents', document_id, 'document_link', 'title', 'summary')

# document_id = '121'
# document_link, title, summary = get_all_info(document_id)
# print("Document Link:", document_link)

# print("Title:", title)
# print("Summary:", summary)


In [196]:
# Example usage
document_id = '211'
document_link = 'https://211.com/document'
title = 'Sample Title211'
summary = 'This is a sample summary.211'

In [197]:
lua_script = """
    if redis.call('sismember', KEYS[1], ARGV[1]) == 1 then
        return 'EXIST'
    else
        redis.call('hset', KEYS[2], ARGV[2], ARGV[1])
        redis.call('hset', KEYS[2], ARGV[3], ARGV[4])
        redis.call('sadd', KEYS[1], ARGV[1])
        return 'OK'
    end
"""
# Run the Lua script
result = r.eval(lua_script, 2, 'document_links', 'documents', document_link, document_id, title, summary)

In [515]:
r.hmget('documents', document_id, 'title', 'summary')

[None, None, None]

In [217]:
r.hset(1, "A", "D")

0

In [219]:
r.hgetall(1)

{b'A': b'D'}

In [231]:
r.sadd('names', 'jupyter')

0

In [232]:
r.sismember('names', 'jupyter')

1

In [233]:
# Save link using
# r.sadd('names', 'jupyter')
# r.sismember('names', 'jupyter')

In [260]:
r.hset('h_employee_profile:nicol', 'Q', '4')

0

In [264]:
r.hgetall("h_employee_profile:nicol")

{b'name': b'Nicol3', b'school': b'X', b'Q': b'4'}

In [265]:
r.hget("h_employee_profile:nicol", 'Q')

b'4'

In [240]:
r.rpush('mylist', 'one', 'two', 'three', 'four', 'five')



5

In [248]:
r.lrange('mylist', 3, 3)


[b'four']

In [239]:
r.lpop('mylist', 3)

In [ ]:
r.rpush('mylist', 'one', 'two', 'three', 'four', 'five')
r.lpop('mylist')
r.lpop('mylist', 2)


In [163]:
get_data('121')

In [514]:
# r.hget('documents', '122')

# Create a new format index

In [68]:
# data = {
#     "document_size": data_json['document_size'],
#     "doc_ids_list": data_json['doc_ids_list']
# }

# data['terms'] = {}

In [69]:
# for keyword in data_json.keys():
#     if keyword not in ['document_size', 'doc_ids_list']:
#         data['terms'][keyword] = data_json[keyword]